# Vector Database Basics

Vector databases help us store, manage, and query the embeddings we created for generative AI, recommenders, and search engines.

Across many of the common use cases, users often find that they need to manage more than just vectors.
To make it easier for practitioners, vector databases should store and manage all of the data they need:
- embedding vectors
- categorical metadata
- numerical metadata
- timeseries metadata
- text / pdf / images / video / point clouds

And support a wide range of query workloads:
- Vector search (may require ANN-index)
- Keyword search (requires full text search index)
- SQL (for filtering)

For this exercise we'll use LanceDB since it's open source and easy to setup

## Creating tables and adding data

Let's create a LanceDB table called `cats_and_dogs` under the local database directory `~/.lancedb`.
This table should have 4 fields:
- the embedding vector
- a string field indicating the species (either "cat" or "dog")
- the breed
- average weight in pounds

We're going to use pydantic to make this easier. First let's create a pydantic model with those fields

In [1]:
from lancedb.pydantic import vector, LanceModel

class CatsAndDogs(LanceModel):
    vector: vector(2)
    species: str
    breed: str
    weight: float

Now connect to a local db at ~/.lancedb and create an empty LanceDB table called "cats_and_dogs"

In [4]:
import lancedb

db = lancedb.connect("~/.lancedb")
table_name = "cats_and_dogs"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=CatsAndDogs)

Let's add some data

First some cats

In [5]:
data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

Now call the `LanceTable.add` API to insert these two records into the table

In [6]:
table.add([dict(d) for d in data])

Let's preview the data

In [7]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


Now let's add some dogs

In [9]:
data = [
    CatsAndDogs(
        vector=[0., 10.],
        species="dog",
        breed="samoyed",
        weight=47.5,
    ),
    CatsAndDogs(
        vector=[0, -1.],
        species="dog",
        breed="corgi",
        weight=26.,
    )
]

In [10]:
table.add([dict(d) for d in data])

In [11]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Querying tables

Vector databases allow us to retrieve data for generative AI applications. Let's see how that's done.

Let's say we have a new animal that has embedding [10.5, 10.], what would you expect the most similar animal will be?
Can you use the table we created above to answer the question?

**HINT** you'll need to use the `search` API for LanceTable and `limit` / `to_df` APIs. For examples you can refer to [LanceDB documentation](https://lancedb.github.io/lancedb/basic/#how-to-search-for-approximate-nearest-neighbors).

In [16]:
table.search([10.5, 10.]).limit(1).to_df()

/var/folders/zl/4wgq9t852p9g7dqy3sc5g07r0000gn/T/ipykernel_81625/1318259148.py:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search([10.5, 10.]).limit(1).to_df()


,vector,species,breed,weight,_distance
0,"[0.0, 10.0]",dog,samoyed,47.5,110.25


Now what if we use cosine distance instead? Would you expect that we get the same answer? Why or why not?

**HINT** you can add a call to `metric` in the call chain

In [17]:
table.search([10.5, 10.]).metric("cosine").limit(1).to_df()

/var/folders/zl/4wgq9t852p9g7dqy3sc5g07r0000gn/T/ipykernel_81625/1416220787.py:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search([10.5, 10.]).metric("cosine").limit(1).to_df()


,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Filtering tables

In practice, we often need to specify more than just a search vector for good quality retrieval. Oftentimes we need to filter the metadata as well.

Please write code to retrieve two most similar examples to the embedding [10.5, 10.] but only show the results that is a cat.

In [19]:
table.search([10.5, 10.]).limit(2).where("species='cat'").to_df()

/var/folders/zl/4wgq9t852p9g7dqy3sc5g07r0000gn/T/ipykernel_81625/318113233.py:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search([10.5, 10.]).limit(2).where("species='cat'").to_df()


,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,190.25


## Creating ANN (Approximate Nearest Neighbor) indices

For larger tables (e.g., >1M rows), searching through all of the vectors becomes quite slow. Here is where the Approximate Nearest Neighbor (ANN) index comes into play. While there are many different ANN indexing algorithms, they all have the same purpose - to drastically limit the search space as much as possible while losing as little accuracy as possible

For this problem we will create an ANN index on a LanceDB table and see how that impacts performance

### First let's create some data

Given the constraints of the classroom workspace, we'll complete this exercise by creating 100,000 vectors with 16D in a new table. Here the embedding values don't matter, so we simply generate random embeddings as a 2D numpy array. We then use the vec_to_table function to convert that in to an Arrow table, which can then be added to the table.

In [20]:
from lance.vector import vec_to_table
import numpy as np

mat = np.random.randn(100_000, 16)
table_name = "exercise3_ann"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, vec_to_table(mat))

### Let's establish a baseline without an index

Before we create the index, let's make sure know what we need to compare against.

We'll generate a random query vector and record it's value in the `query` variable so we can use the same query vector with and without the ANN index.

In [21]:
query = np.random.randn(16)
table.search(query).limit(10).to_df()

/var/folders/zl/4wgq9t852p9g7dqy3sc5g07r0000gn/T/ipykernel_81625/229240492.py:2: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


,vector,_distance
0,"[-1.2067305, 0.4210034, -0.6656878, 0.01206438...",2.661047
1,"[-0.34648255, -0.48921132, -0.8546587, 0.67892...",3.841872
2,"[-0.22639008, 0.022125052, -0.14792703, 0.5289...",3.962211
3,"[-1.1997696, 0.05737256, -0.17494711, 0.783640...",4.009890
4,"[-0.738016, 0.4685821, -0.19721188, 0.8807055,...",4.048850
5,"[-0.08298535, 0.036312394, -0.7022423, -0.0956...",4.319125
6,"[-0.29336548, 0.45664907, -0.23120388, 0.64135...",4.588379
7,"[-0.33223477, -0.76999444, 0.0361639, 0.964591...",4.811741
8,"[-0.4636814, -0.22055928, -0.67725873, -0.3969...",4.843414
9,"[-1.3230187, -0.13957784, -0.56273735, 0.33700...",4.938725


Please write code to compute the average latency of this query

In [25]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

2.23 ms ± 79.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Now let's create an index

There are many possible index types ranging from hash based to tree based to partition based to graph based.
For this task, we'll create an IVFPQ index (partition-based index with product quantization compression) using LanceDB.

Please create an IVFPQ index on the LanceDB table such that each partition is 4000 rows and each PQ subvector is 8D.

**HINT** 
1. Total vectors / number of partitions = number of vectors in each partition
2. Total dimensions / number of subvectors = number of dimensions in each subvector
3. This step can take about 7-10 minutes to process and execute in the classroom workspace.

In [26]:
table.create_index(num_partitions=256, num_sub_vectors=16)

Now let's search through the data again. Notice how the answers now appear different.
This is because an ANN index is always a tradeoff between latency and accuracy.

In [27]:
table.search(query).limit(10).to_df()

/var/folders/zl/4wgq9t852p9g7dqy3sc5g07r0000gn/T/ipykernel_81625/4174343316.py:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


,vector,_distance
0,"[-1.2067305, 0.4210034, -0.6656878, 0.01206438...",2.654516
1,"[-0.34648255, -0.48921132, -0.8546587, 0.67892...",3.836117
2,"[-0.22639008, 0.022125052, -0.14792703, 0.5289...",3.969935
3,"[-1.1997696, 0.05737256, -0.17494711, 0.783640...",3.989986
4,"[-0.738016, 0.4685821, -0.19721188, 0.8807055,...",4.025091
5,"[-0.08298535, 0.036312394, -0.7022423, -0.0956...",4.328153
6,"[-0.29336548, 0.45664907, -0.23120388, 0.64135...",4.603116
7,"[-0.33223477, -0.76999444, 0.0361639, 0.964591...",4.803634
8,"[-0.4636814, -0.22055928, -0.67725873, -0.3969...",4.849376
9,"[-1.3230187, -0.13957784, -0.56273735, 0.33700...",4.923937


Now write code to compute the average latency for querying the same table using the ANN index.

**SOLUTION** The index is implementation detail, so it should just be running the same code as above. You should see almost an order of magnitude speed-up. On larger datasets, this performance difference should be even more pronounced.

In [28]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

650 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Deleting rows

Like with other kinds of databases, you should be able to remove rows from the table.
Let's go back to our tables of cats and dogs

In [29]:
table = db["cats_and_dogs"]

In [30]:
len(table)

4

Can you use the `delete` API to remove all of the cats from the table?

**HINT** use a SQL like filter string to specify which rows to delete from the table

In [31]:
table.delete("species='cat'")

In [32]:
len(table)

2

## What if I messed up?

Errors is a common occurrence in AI. What's hard about errors in vector search is that oftentimes a bad vector doesn't cause a crash but just creates non-sensical answers. So to be able to rollback the state of the database is very important for debugging and reproducibility

So far we've accumulated 4 actions on the table:
1. creation of the table
2. added cats
3. added dogs
4. deleted cats

What if you realized that you should have deleted the dogs instead of the cats?

Here we can see the 4 versions that correspond to the 4 actions we've done

In [33]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 7, 21, 15, 58, 24, 561574),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2024, 7, 21, 16, 0, 35, 180614),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2024, 7, 21, 16, 1, 53, 409058),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2024, 7, 22, 12, 42, 20, 113174),
  'metadata': {}}]

Please write code to restore the version still containing the whole dataset

In [34]:
table = db["cats_and_dogs"]

In [35]:
len(table)

2

In [36]:
table.restore(3)

In [37]:
table.delete("species='dog'")

In [38]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 7, 21, 15, 58, 24, 561574),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2024, 7, 21, 16, 0, 35, 180614),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2024, 7, 21, 16, 1, 53, 409058),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2024, 7, 22, 12, 42, 20, 113174),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2024, 7, 22, 12, 45, 3, 358799),
  'metadata': {}},
 {'version': 6,
  'timestamp': datetime.datetime(2024, 7, 22, 12, 45, 17, 412356),
  'metadata': {}}]

In [39]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


## Dropping a table

You can also choose to drop a table, which also completely removes the data.
Note that this operation is not reversible.

In [40]:
"cats_and_dogs" in db

True

Write code to irrevocably remove the table "cats_and_dogs" from the database

In [41]:
db.drop_table("cats_and_dogs")

How would you verify that the table has indeed been deleted?

In [42]:
table.name in db

False

## Summary

Congrats, in this exercise you've learned the basic operations of vector databases from creating tables, to adding data, and to querying the data. You've learned how to create indices and you saw first hand how it changes the performance and the accuracy. Lastly, you've learned how to debug and rollback when errors happen.